# Exploratory Data Analysis of Reviews Data - EDA

The major objective of this notebook is to perform an Exploratory Data Analysis, also known as EDA, in the data related to games reviews. To do that, we start by joining our parquet files into a dataset. After, we ensure that each variable is in the correct format and start doing some simple explorations.

The structure of this notebook is as follows:

[0. Import Libraries](#libraries) <br>

# 0. Import Libraries<a id='libraries'></a>
[to the top](#top)  

The first step is to import the necessary libraries.
These imports cover the necessary libraries for working with Parquet files, data manipulation, visualization, and text preprocessing.

In [1]:
import pyarrow.parquet as pq
import os
import pandas as pd
from pathlib import Path
import re
from datetime import datetime, timezone
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix 

Concate Parquets

In [2]:
have_df = False

In [3]:
def concate_parquet(directory_path):

    # Get a list of Parquet files in the directory
    parquet_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.parquet')]

    # Initialize an empty list to store DataFrames
    dfs = []

    # Iterate through each Parquet file
    for file in parquet_files:
        # Read the Parquet file into a DataFrame
        df = pq.read_table(file).to_pandas()
        
        # Extract the substring before the underscore from the filename
        filename = os.path.basename(file)
        appid = filename.split('_')[0]
        
        # Add a new column with the extracted prefix value
        df['AppId'] = appid
        
        # Append the modified DataFrame to the list
        dfs.append(df)

    # Concatenate all DataFrames into one
    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df


In [4]:
if not have_df:
    parquets_df = concate_parquet('./data/parquets/')
    parquets_df

Column author contains a dictionary

Transform the dictionary into columns of the dataframe

In [5]:
def expand_column(concatenated_df):
    # Use json_normalize to expand the dictionary column into separate columns
    expanded_df = pd.json_normalize(concatenated_df['author'])

    # Concatenate the expanded DataFrame with the original DataFrame
    result_df = pd.concat([concatenated_df, expanded_df], axis=1)

    # Drop the original dictionary column
    result_df.drop('author', axis=1, inplace=True)

    return result_df

In [6]:
if not have_df:
    parquets_df = expand_column(parquets_df)
    parquets_df

Save dataframe

In [7]:
if not have_df:
    parquets_df['recommendationid'].duplicated().sum() # Cant be index

In [8]:
if not have_df:
    parquets_df.to_csv('parquets_df.csv', index=False)

OSError: [Errno 28] No space left on device

In [ ]:
if have_df:
    parquets_df = pd.read_csv("parquets_df.csv")
    parquets_df

## EDA

### Display the basic information about the DataFrame
To get a concise summary of the DataFrame.
This helps in understanding the structure of the data, identifying any issues such as missing values or incorrect data types, and getting an overall sense of the dataset.

In [ ]:
# Display the basic information about the DataFrame
print("DataFrame Info:")
print(parquets_df.info())

### Check for Missing Values
Understanding which columns have missing values helps in deciding how to handle them, such as filling them with appropriate values or dropping rows/columns.

In [ ]:
# Check for missing values
print("\nMissing Values:")
print(parquets_df.isnull().sum()) # calculates the total number of missing values in each column of the DataFrame.

### Check for duplicates

Duplicate rows can skew the analysis and should be addressed by either removing them or keeping them based on the context of the data.

In [ ]:
parquets_df.duplicated() # returns a boolean value (true for duplicates and false for no duplicates)

### Datatype changes
To ensure that each column has the correct data type for analysis.
Converting data types can improve efficiency and accuracy of operations, and ensure consistency in data representation.

In [ ]:
def format_datatypes(gamereviews):
    # Handle missing values in the review column
    gamereviews["review"] = gamereviews["review"].fillna("")
    
    # Format "timestamp_created" to int
    gamereviews["timestamp_created"] = gamereviews["timestamp_created"].astype('int64')
    
    # Format timestamps
    gamereviews["timestamp_created"] = pd.to_datetime(gamereviews["timestamp_created"], unit='s', utc=True)
    gamereviews["timestamp_updated"] = pd.to_datetime(gamereviews["timestamp_updated"], unit='s', utc=True)
    gamereviews["last_played"] = pd.to_datetime(gamereviews["last_played"], unit='s', utc=True)
    
    # Remove unnecessary fields
    gamereviews.drop(columns=["language", "hidden_in_steam_china", "steam_china_location"], inplace=True)
    
    # Format boolean fields
    gamereviews["voted_up"] = gamereviews["voted_up"].astype(bool)
    gamereviews["steam_purchase"] = gamereviews["steam_purchase"].astype(bool)
    gamereviews["received_for_free"] = gamereviews["received_for_free"].astype(bool)
    gamereviews["written_during_early_access"] = gamereviews["written_during_early_access"].astype(bool)
    gamereviews["weighted_vote_score"] = gamereviews["weighted_vote_score"].astype(float)
    
    return gamereviews

In [ ]:
parquets_df = format_datatypes(parquets_df)
parquets_df

Timestamp columns (timestamp_created, timestamp_updated, last_played) were converted to datetime objects using the pd.to_datetime() function to facilitate temporal analysis. Unnecessary fields such as language, hidden_in_steam_china, and steam_china_location were removed from the DataFrame to streamline the dataset. Boolean fields (voted_up, steam_purchase, received_for_free, written_during_early_access) were converted to boolean data types for clear representation. Additionally, the weighted_vote_score column was converted to a float data type to accurately reflect its numerical nature. These data type changes ensure that our DataFrame is ready for comprehensive analysis and modeling.

### Numerical Variables
To isolate numerical variables from the DataFrame for further analysis.
This step helps in focusing on numerical features that might be relevant for modeling or exploration.

In [ ]:
parquets_df.columns

In [ ]:
# Select numerical variables
# Remove Id variables
numerical_df = parquets_df.select_dtypes(include=['int64', 'float64']).drop(['recommendationid', 'AppId', 'steamid'], axis=1)

### Summary Statistics
Summary statistics provide insights into the central tendency, dispersion, and shape of the distribution of each numerical variable.

In [ ]:
# Get to know the dataset
numerical_df.describe().T

### Correlation Matrix
To explore the linear relationship between pairs of numerical variables in the DataFrame.
Helps in understanding how variables are related to each other, which can guide feature selection, dimensionality reduction, or predictive modeling.

In [ ]:
# Calculate correlations between numerical columns
correlation_matrix = numerical_df.corr()

# Visualize the correlation matrix as a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()


The heatmap produced visually represents the strength and direction of the linear relationship between pairs of numerical variables. Correlation coefficients close to 1 indicate a strong positive linear relationship, while coefficients close to -1 indicate a strong negative linear relationship. Coefficients close to 0 suggest little to no linear relationship.
High correlations (either positive or negative) between variables may indicate redundancy or multicollinearity, which can affect the performance of predictive models.

### Scatter Matrix of User's Data
To visualize relationships between multiple numerical variables in the user's data.
Scatter matrices provide a comprehensive view of pairwise relationships between variables, along with histograms of individual variables along the diagonal.

In [ ]:
# Create a scatter matrix
scattermatrix = scatter_matrix(parquets_df[['num_games_owned', 'num_reviews', 'playtime_forever', 'playtime_last_two_weeks', 'playtime_at_review']],
                                diagonal='hist', figsize=(15,10))

# Get the number of columns in the matrix
n = scattermatrix.shape[1]

# Mask the upper triangle of the matrix
for i in range(n):
    for j in range(i + 1, n):
        scattermatrix[i, j].set_visible(False)
        
# Rotate the axis labels
for ax in scattermatrix.ravel():    
    ax.yaxis.label.set_rotation(0)
    ax.xaxis.label.set_rotation(90)

# Choose a title and make the title appear closer to the graph
plt.suptitle("Scatter Matrix of Users Data", y = 0.93)
    
# Show the plot
plt.show()

### Scatter Matrix of Reviews' Data
To visualize relationships between multiple numerical variables in the reviews' data.
Similar to the previous scatter matrix, this visualization provides insights into pairwise relationships between variables and their distributions.

In [ ]:
# Create a scatter matrix
scattermatrix = scatter_matrix(parquets_df[['votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count']],
                                diagonal='hist', figsize=(15,10))

# Get the number of columns in the matrix
n = scattermatrix.shape[1]

# Mask the upper triangle of the matrix
for i in range(n):
    for j in range(i + 1, n):
        scattermatrix[i, j].set_visible(False)
        
# Rotate the axis labels
for ax in scattermatrix.ravel():    
    ax.yaxis.label.set_rotation(0)
    ax.xaxis.label.set_rotation(90)

# Choose a title and make the title appear closer to the graph
plt.suptitle("Scatter Matrix of Reviews' Data", y = 0.93)
    
# Show the plot
plt.show()

### Time-Based Variables
To visualize the distribution of the timestamp_created variable, which represents the time when reviews were created.
Understanding the distribution of timestamps can provide insights into temporal patterns or trends in review creation.

In [ ]:
# Distribution of timestamp_created
plt.figure(figsize=(12, 8))
sns.histplot(data=parquets_df['timestamp_created'], bins=30, kde=True)
plt.title("Distribution of Timestamp Created")
plt.xlabel("Timestamp Created")
plt.ylabel("Frequency")
plt.show()

Peaks in the distribution indicate periods of high review activity.

### Filtered Distribution of Timestamp Created (2022-2024)
To filter the data for timestamps between 2022 and 2024 and visualize the distribution of review creation timestamps during this period.
Additionally, to analyze the distribution of game contributions (reviews) for each game during the same period.

In [ ]:
# Filter the data for timestamps between 2022 and 2024 - Change dates using the peaks of the data
start_date = '2020-01-01'
end_date = '2024-12-31'
filtered_df = parquets_df[(parquets_df['timestamp_created'] >= start_date) & (parquets_df['timestamp_created'] <= end_date)]

# Plot the distribution of timestamp_created between 2022 and 2024
plt.figure(figsize=(12, 8))
sns.histplot(data=filtered_df['timestamp_created'], bins=30, kde=True)
plt.title("Distribution of Timestamp Created (2022-2024)")
plt.xlabel("Timestamp Created")
plt.ylabel("Frequency")
plt.show()

# Plot the distribution of game contributions in the same period
plt.figure(figsize=(12, 8))
sns.countplot(data=filtered_df, y='AppId', order=filtered_df['AppId'].value_counts().index)
plt.title("Game Contributions (2022-2024)")
plt.xlabel("Frequency")
plt.ylabel("AppId")
plt.show()

The first plot visualizes the frequency distribution of review creation timestamps between 2022 and 2024. This helps in identifying any temporal patterns or trends in review activity during this period.
The second plot displays the distribution of game contributions (reviews) for each game during the same period, allowing for the identification of popular games or those with the highest number of reviews.

### Time Series Analysis - Monthly Count of Voted Up Reviews
To perform time series analysis on the count of "voted up" reviews over monthly intervals, this helps in understanding patterns, trends, and seasonality in the data over time.

In [ ]:
# Time series analysis
parquets_df.set_index('timestamp_created', inplace=True)
plt.figure(figsize=(12, 8))
parquets_df['voted_up'].resample('M').count().plot()
plt.title("Monthly Count of Voted Up Reviews")
plt.xlabel("Date")
plt.ylabel("Count")
plt.show()

### Review Analysis - Filtered "EDIT:" Reviews
To filter reviews that start with "EDIT:" and analyze their count.

In [ ]:
# Filter reviews that start with "EDIT:"
edit_reviews = parquets_df[parquets_df['review'].str.startswith('EDIT:')]
print(len(edit_reviews))

The count of reviews that start with "EDIT:" provides insights into the frequency of edited reviews, which may indicate user interactions or updates to previous reviews

### Review Length Histogram
To visualize the distribution of review lengths using a histogram.

In [ ]:
# Histogram of Review Lengths
parquets_df['review_length'] = parquets_df['review'].apply(len)
plt.figure(figsize=(10, 6))
plt.hist(parquets_df['review_length'], bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram of Review Lengths')
plt.xlabel('Review Length')
plt.ylabel('Frequency')
plt.show()

The histogram of review lengths visualizes the distribution of review lengths, allowing for the identification of common or uncommon review lengths and their frequencies.

### Top 10 Games with the Longest Preprocessed Review Lengths
To identify and visualize the top 10 games with the longest preprocessed review lengths.

In [ ]:
top_10_longest = parquets_df.sort_values(by='review_length', ascending=True).tail(10)

# Plotting the bar plot
plt.figure(figsize=(10, 6))
plt.bar(top_10_longest['AppId'], top_10_longest['review_length'], color='skyblue') # Change AppID to Games name
plt.xlabel('ID')
plt.ylabel('Length of Preprocessed Review')
plt.title('Length of Preprocessed Review by ID')
plt.show()

The DataFrame is sorted based on the review length in descending order to identify the games with the longest preprocessed review lengths. A horizontal bar plot is generated using Matplotlib to visualize the review lengths of these top 10 games.

### Distribution of Response Time in Days
To calculate the response time in days between the timestamp created and the timestamp when the developer responded. To check for any NaN values in the response time and drop rows with NaN values.

In [ ]:
# Convert timestamps to datetime objects and make them timezone-naive
parquets_df['timestamp_dev_responded'] = pd.to_datetime(parquets_df['timestamp_dev_responded']).dt.tz_localize(None)
parquets_df['timestamp_created'] = pd.to_datetime(parquets_df['timestamp_created']).dt.tz_localize(None)

# Calculate response time in days
parquets_df['response_time'] = (parquets_df['timestamp_created'] - parquets_df['timestamp_dev_responded']).dt.days

# Check for any NaN values in response_time
print(parquets_df['response_time'].isna().sum(), "NaN values in response_time")

# Drop rows with NaN response_time if any
parquets_df = parquets_df.dropna(subset=['response_time'])

# Distribution of Response Time
plt.figure(figsize=(12, 6))
sns.histplot(parquets_df['response_time'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Response Time in Days')
plt.xlabel('Response Time (days)')
plt.ylabel('Frequency')
plt.show()

The histogram displays the distribution of response time in days, indicating how quickly developers responded to user reviews.
Analyzing the distribution can provide insights into the typical response time and any outliers or patterns in developer responsiveness.

### Sentiment Analysis using VADER
To calculate the sentiment polarity of each review using VADER, visualize the distribution of review sentiment polarity and explore its relationship with the number of votes up.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to calculate sentiment polarity using VADER
def get_vader_sentiment(review):
    return sid.polarity_scores(review)['compound']

# Calculate sentiment polarity for each review
parquets_df['sentiment'] = parquets_df['review'].apply(get_vader_sentiment)

# Distribution of Sentiment
plt.figure(figsize=(18, 6))  # Changed the width to 18 for a wider plot
sns.histplot(parquets_df['sentiment'], bins=30, kde=True, color='purple')
plt.title('Distribution of Review Sentiment')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Frequency')
plt.show()

# Sentiment vs. Votes Up
plt.figure(figsize=(18, 6))  # Changed the width to 18 for a wider plot
sns.scatterplot(x='sentiment', y='votes_up', data=parquets_df, alpha=0.5)
plt.title('Sentiment vs. Votes Up')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Votes Up')
plt.show()

The histogram provides insights into the distribution of sentiment polarity across all reviews, indicating the prevalence of positive, negative, or neutral sentiments.
The scatter plot explores the relationship between sentiment polarity and the number of votes up, allowing for the examination of any patterns or correlations between review sentiment and user engagement.

### Analyzing the Relationship Between Response Time and Votes Up
To explore the relationship between response time (in days) and the number of votes up received by reviews.

In [ ]:
# Calculate response time in days
parquets_df['response_time'] = (pd.to_datetime(parquets_df['timestamp_dev_responded']) - pd.to_datetime(parquets_df['timestamp_created'])).dt.days

# Response Time vs. Votes Up
plt.figure(figsize=(12, 6))
sns.scatterplot(x='response_time', y='votes_up', data=parquets_df, alpha=0.5)
plt.title('Response Time vs. Votes Up')
plt.xlabel('Response Time (days)')
plt.ylabel('Votes Up')
plt.show()

The scatter plot allows for the examination of any correlations or patterns between response time and user engagement, as indicated by the number of votes up received by reviews.
By analyzing the distribution of points, we can identify whether there is a relationship between response time and user engagement. For example, you may observe that reviews with shorter response times tend to receive more votes up, indicating higher user satisfaction or engagement.
Understanding this relationship can help developers or platform administrators prioritize timely responses to reviews to enhance user satisfaction and engagement.

### Text Preprocessing Function
To preprocess text data by removing noise and irrelevant information, making it suitable for natural language processing tasks such as sentiment analysis or text classification.

In [ ]:
def preprocess_text(text):
    '''Takes a string as input and returns a preprocessed string as output'''
    
    #remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)

    #lowercase
    text = [word.lower() for word in text]
    
    #remove stopwords
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    
    #remove numbers
    text = [word for word in text if word.isalpha()]

    # Join
    text = ' '.join(text)

    #remove isolated consonants:
    text = re.sub(r'\b([^aeiou-])\b',' ',text)

    return text

### Word Cloud Visualization of Preprocessed Text Data
To visually explore the most frequent words in the preprocessed text data, providing insights into the common themes or topics present in the reviews.

In [ ]:
# Apply the preprocess_text function to each sentence in the DataFrame
parquets_df['review_preprocessed'] = parquets_df['review'].apply(preprocess_text)

# Join the preprocessed sentences into a single string
wordcloud_input = ' '.join(parquets_df['review_preprocessed'])

# Create a wordcloud
wordcloud = WordCloud(width = 800, height = 600, 
                background_color ='white', 
                max_font_size = 50, 
                colormap='Dark2').generate(wordcloud_input)

# Plot the WordCloud image
plt.figure(figsize = (8, 6), facecolor = None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

The word cloud allows for a quick understanding of the predominant themes or topics present in the reviews.
Words that appear larger and more prominently in the word cloud are those that occur frequently across multiple reviews.
Analyzing the word cloud can reveal insights into the sentiments, preferences.

### Pie Chart: Voted Up vs. Voted Down Reviews
To visualize the distribution of reviews based on whether they were voted up or down by users, providing an overview of user sentiment towards the reviewed content.

In [ ]:
# Pie Chart of Voted Up vs. Voted Down Reviews
voted_counts = parquets_df['voted_up'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(voted_counts, labels=voted_counts.index, autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
plt.title('Voted Up vs. Voted Down Reviews')
plt.show()

A larger portion of reviews in the "Voted Up" category suggests that the majority of users have positive sentiments towards the reviewed content.
